In [1]:
import os
import json

from promptflow.core import AzureOpenAIModelConfiguration, Prompty

# Initialize Azure OpenAI Connection
model_config = AzureOpenAIModelConfiguration(
        azure_deployment="gpt-4",
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
    )

In [4]:
import sys
sys.path.append('../contoso_chat')  # Replace '/path/to/contoso_chat' with the actual path to the 'contoso_chat' folder

from chat_request import get_response


## Get output from data and save to results jsonl file

In [5]:
import pandas as pd

data_path = "../data/data.jsonl"

df = pd.read_json(data_path, lines=True)
df.head()

,customerId,question,chat_history,intent
0,4,tell me about your hiking jackets,[],chat
1,1,Do you have any climbing gear?,[],chat
2,3,Can you tell me about your selection of tents?,[],chat
3,6,Do you have any hiking boots?,[],chat
4,2,What gear do you recommend for hiking?,[],chat


In [8]:

results = []

for index, row in df.iterrows():
    customerId = row['customerId']
    question = row['question']
    
    # Run contoso-chat/chat_request flow to get response
    response = get_response(customerId=customerId, question=question, chat_history=[])
    
    # Add results to list
    result = {
        'question': question,
        'context': response["context"],
        'answer': response["answer"]
    }
    results.append(result)

# Save results to a JSONL file
with open('result.jsonl', 'w') as file:
    for result in results:
        file.write(json.dumps(result) + '\n')

inputs: 4 tell me about your hiking jackets
context: [{'id': '17', 'title': 'RainGuard Hiking Jacket', 'content': "Introducing the MountainStyle RainGuard Hiking Jacket - the ultimate solution for weatherproof comfort during your outdoor undertakings! Designed with waterproof, breathable fabric, this jacket promises an outdoor experience that's as dry as it is comfortable. The rugged construction assures durability, while the adjustable hood provides a customizable fit against wind and rain. Featuring multiple pockets for safe, convenient storage and adjustable cuffs and hem, you can tailor the jacket to suit your needs on-the-go. And, don't worry about overheating during intense activities - it's equipped with ventilation zippers for increased airflow. Reflective details ensure visibility even during low-light conditions, making it perfect for evening treks. With its lightweight, packable design, carrying it inside your backpack requires minimal effort. With options for men and women,

In [10]:
# load apology evaluatorfrom prompty
groundedness_eval = Prompty.load(source="prompty/groundedness.prompty", model={"configuration": model_config})
fluency_eval = Prompty.load(source="prompty/fluency.prompty", model={"configuration": model_config})
coherence_eval = Prompty.load(source="prompty/coherence.prompty", model={"configuration": model_config})
relevance_eval = Prompty.load(source="prompty/relevance.prompty", model={"configuration": model_config})

In [14]:
# Evaluate results from results file
results_path = 'result.jsonl'
results = []
with open(results_path, 'r') as file:
    for line in file:
        results.append(json.loads(line))

for result in results:
    question = result['question']
    context = result['context']
    answer = result['answer']
    
    groundedness_score = groundedness_eval(question=question, answer=answer, context=context)
    fluency_score = fluency_eval(question=question, answer=answer, context=context)
    coherence_score = coherence_eval(question=question, answer=answer, context=context)
    relevance_score = relevance_eval(question=question, answer=answer, context=context)
    
    result['groundedness'] = groundedness_score
    result['fluency'] = fluency_score
    result['coherence'] = coherence_score
    result['relevance'] = relevance_score

# Save results to a JSONL file
with open('result_evaluated.jsonl', 'w') as file:
    for result in results:
        file.write(json.dumps(result) + '\n')

# Print results

df = pd.read_json('result_evaluated.jsonl', lines=True)
df.head()

Question: tell me about your hiking jackets
Question: Do you have any climbing gear?
Question: Can you tell me about your selection of tents?
Question: Do you have any hiking boots?
Question: What gear do you recommend for hiking?
Question: what is the temperature rating of my sleeping bag?
Question: what is the temperature rating of the cozynights sleeping bag?
Question: what is the waterproof rating of the tent I bought?
Question: what is the waterproof rating of the TrailMaster X4 Tent's rainfly?
Question: What is your return or exchange policy?
Question: is the jacket I bought machine washable?
